In [3]:
from flask import Flask, redirect, url_for, flash, request, render_template, copy_current_request_context
from decouple import config
import boto3
import requests
import hashlib
import pandas as pd
import cv2
import json
import hashlib
import scipy.misc
from PIL import Image
from io import BytesIO
from werkzeug.utils import secure_filename

In [4]:
def get_matching_s3_keys(bucket, prefix='', suffix=''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    kwargs = {'Bucket': bucket, 'Prefix': prefix}
    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.endswith(suffix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [8]:
results = get_matching_s3_keys(S3_BUCKET)

In [260]:
S3_KEY = '***'
S3_SECRET = '***'
S3_BUCKET = 'omega2020'
ExtraArgs='{"ACL": "public-read", "ContentType": "image/png", "ContentDisposition": "inline"}'

In [10]:
s3.list_objects_v2(Bucket='example-bukkit')

NameError: name 's3' is not defined

In [15]:
s3.list_objects_v2

<bound method ClientCreator._create_api_method.<locals>._api_call of <botocore.client.S3 object at 0x7fe762e35fd0>>

In [16]:
def get_matching_s3_keys(bucket, prefix='', suffix=''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    kwargs = {'Bucket': bucket, 'Prefix': prefix}
    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.endswith(suffix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [35]:
start_url = 'https://omega2020.s3.amazonaws.com/raw_puzzles/'

test = get_matching_s3_keys(S3_BUCKET, 'raw_test', '.png')

clean_urls = []
urls = list(test)
for url in urls:
    url = str(url)
    new_url = url.replace(" ","+")
    new_url = start_url + new_url
    clean_urls.append(new_url)

In [36]:
clean_urls[0]

'https://omega2020.s3.amazonaws.com/raw_puzzles/raw_test/Screenshot+from+2020-01-12+11-45-05.png'

In [44]:
import glob
#print(glob.glob("cell_data/*_bulk_processed.png"))

In [55]:
#for file in glob.glob("cell_data/*_bulk_processed.png"):
#    file = file.split("/")[1]
#    print(file)
    

In [58]:
test = glob.glob("cell_data/*_bulk_processed.png")

In [59]:
""" Moving all files of one format from one folder to another"""
# import shutil
# import os

# source = 'cell_data/'
# dest1 = 'cell_data/processed_cells'


# moving = []
# for file in glob.glob("cell_data/*_bulk_cell.png"):
#     file = file.split("/")[1]
#     moving.append(file)
    
# for f in moving:
#         shutil.move(source+f, dest1)
        

In [62]:
os.getcwd()

'/home/rob/Desktop/lambda/labs/github/Notebooks'

In [111]:
import numpy as np 
from skimage import io
#for f in glob.glob("cell_data/processed_cells/*"):
    img = io.imread(f)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray.shape

In [213]:
test_cell = list(glob.glob("cell_data/processed_cells/*"))

In [221]:
cell_arrays = np.zeros((784,))
from tempfile import TemporaryFile

def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

split_cells = divide_chunks(test_cell,10000)

i = 0
for small_cell in split_cells:
    cell_arrays = np.zeros((784,))
    for f in small_cell:
        img = io.imread(f)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = gray.flatten()
        cell_arrays = np.append(cell_arrays,gray,axis=0)
    i = i+1
    cell_arrays = cell_arrays.reshape( (len(small_cell)+1) , 784)
    np.save('all_cell_arrays_'+str(i)+'.npy', cell_arrays)

In [232]:
test_load = np.load('all_cell_arrays_9.npy')[1234]
test_load = test_load.reshape(28,28)

In [234]:
test_load.shape

(28, 28)

In [184]:
len(cell_arrays)

4

In [192]:
for i in range(len(cell_arrays)):
    img_array = cell_arrays[i]
    img_array = img_array.reshape(28,28)
    new_im = Image.fromarray(img_array)
    new_im.show()

In [162]:
for im in cell_arrays:
    new_im = Image.fromarray(im)
    print(new_im)

IndexError: tuple index out of range

In [243]:
new_im = Image.fromarray(test_load)


In [245]:
new_im.show()

In [246]:
os.getcwd()

'/home/rob/Desktop/lambda/labs/github/Notebooks'

In [248]:
all_arrays = list(glob.glob("*.npy"))

In [249]:
all_arrays

['all_cell_arrays_5.npy',
 'all_cell_arrays_6.npy',
 'all_cell_arrays_7.npy',
 'all_cell_arrays_8.npy',
 'all_cell_arrays_4.npy',
 'all_cell_arrays_1.npy',
 'all_cell_arrays_9.npy',
 'all_cell_arrays_2.npy',
 'all_cell_arrays_3.npy']

In [306]:
arrays = cell_arrays = np.zeros((1,784))
for array in all_arrays:
    a = np.load(array)
    arrays = np.append(arrays,a,axis=0)
    

In [307]:
arrays

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [258]:
np.save('all_cell_arrays_TOTAL.npy', arrays)

In [261]:
arrays[0].

(784,)

In [309]:
unique_array = np.unique(arrays, axis=0)

In [310]:
unique_array.shape

(68766, 784)

In [311]:
np.save('all_cell_arrays_unique.npy',unique)

In [313]:
class_array = np.zeros((unique_array.shape[0],))

In [314]:
class_array.shape

(68766,)

In [315]:
class_array = np.where(class_array==0, 999, class_array) 

In [316]:
cell_array = sorted_array

In [317]:
cell_array.shape

(68766, 784)

In [318]:
class_array.shape

(68766,)

In [319]:
np.save('classification_array.npy',class_array)